### Preliminaries (See Scrape.ipynb for more details)

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver import ActionChains as AC   
from selenium.webdriver.common.keys import Keys  
import time
from tqdm.notebook import tqdm

opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False) 
opciones.add_argument('--start-maximized')
#opciones.add_argument('--incognito')


In [9]:
PATH=ChromeDriverManager().install()
driver=webdriver.Chrome(PATH, options=opciones) 

C:\Users\jorge\AppData\Local\Temp\ipykernel_4476\2478329780.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH, options=opciones)


In [10]:
#find element
f = lambda x, y=driver : y.find_element(By.XPATH, x)
#find elements
fs = lambda x, y=driver : y.find_elements(By.XPATH, x)
#find and prepare to click
fc = lambda x, y=driver : y.find_element(By.XPATH, x).click()

### List of Best Selling Artists

##### Open Page

In [11]:
top_artists = 'https://en.wikipedia.org/wiki/List_of_best-selling_music_artists'
driver.get(top_artists)
time.sleep(2)

##### Headers

In [12]:
headers = [j.text for j in fs('//*[@id="mw-content-text"]/div[1]/table[1]/thead/tr/th')]
headers[-2] = headers[-2].split('\n')[0]

##### Body

In [13]:
all_tables = [[[f('th',i).text] +[j.text for j in fs('td',i)] for i in fs('tbody/tr',k)] for k in fs('//*[@id="mw-content-text"]/div//table')[:5]]
values = [rows for table in all_tables for rows in table]
artists = pd.DataFrame(values, columns = headers)

##### Cleaning Data

In [14]:
for i in ['Period active','Release-year of first charted record','Genre','Claimed sales']:
    artists[i] = artists[i].apply(lambda x : x.split('[')[0])

artists['Total certified units'] = artists['Total certified units'].apply(lambda x : x.strip('show\n\n'))
artists['Country'] = artists['Country'].apply(lambda x : x.split('\n')[0])
artists['Active'] = artists['Period active'].apply(lambda x :'present' in x.lower())
artists['Total certified units'] = artists['Total certified units'].apply( lambda x : float(x.split('m')[0]) * 1000000)
artists['Claimed sales'] = artists['Claimed sales'].apply( lambda x : float(x.split('m')[0]) * 1000000)

artists.to_parquet('../data/wikitop100list/artists.parquet',index=False)

##### Exporting Genres

In [15]:
genres = set()
for i in artists['Genre'].unique():
    x = i.split('/')
    for j in x:
        j = j.strip()
    for j in x:
        genres.add(j.lower())
Genres = pd.DataFrame(list(genres),columns=['Genre'])
Genres['genre_id'] = list(range(1,len(Genres)+1))
Genres.to_parquet('../data/wikitop100list/genres.parquet',index=False)

#### Creating Relation Genres-Artists

In [16]:
artist_genre = pd.DataFrame(artists['Artist'])
for j,i in enumerate(genres):
    artist_genre[j+1] = artists['Genre'].apply(lambda x : i in x.lower())

artist_genre['ids'] = ' '
for l,i in enumerate(artist_genre.itertuples()):
    genre_list = []
    for k,j in enumerate(i[2:-1]):
        if j:
            genre_list.append(k+1)
    artist_genre.at[ l , 'ids'] = genre_list
artist_genre = artist_genre[['Artist','ids']]
artist_genre = artist_genre.explode('ids')
artist_genre.to_parquet('../data/wikitop100list/artist_genre.parquet',index=False)

In [17]:
x = pd.read_parquet('../data/wikitop100list/artists.parquet')
x

,Artist,Country,Period active,Release-year of first charted record,Genre,Total certified units,Claimed sales,Active
0,The Beatles,United Kingdom,1960–1970,1962,Rock / pop,290600000.0,600000000.0,False
1,Elvis Presley,United States,1953–1977,1956,Rock and roll / pop / country,231800000.0,500000000.0,False
2,Michael Jackson,United States,1964–2009,1971,Pop / rock / dance / soul / R&B,279900000.0,400000000.0,False
3,Elton John,United Kingdom,1962–present,1970,Pop / rock,208200000.0,300000000.0,True
4,Queen,United Kingdom,1971–present,1973,Rock,184200000.0,300000000.0,True
...,...,...,...,...,...,...,...,...
97,The Black Eyed Peas,United States,1995–present,1998,Hip-hop / R&B / dance,69800000.0,80000000.0,True
98,Van Halen,United States,1978–2020,1978,Hard rock / heavy metal,65200000.0,80000000.0,False
99,Ayumi Hamasaki,Japan,1998–present,1998,J-pop / pop / dance / electronic,64500000.0,80000000.0,True
100,Tom Petty,United States,1976–2017,1977,Rock,36000000.0,80000000.0,False
